# Quantifier Elimination for Real Closed Fields by Cylindrial Algebraic Decomposition

　この論文は, CAD法やQEの計算理論に関する解説やそれらの解放手法を提案している論文を網羅的に集めた論文集"Quantifier Elimination and Cylindrical Algebraic Decomposition"(1996)に掲載された, George E. Collinsによる現論文である.

　このノートブックは, CADの原理的な理解と実装を目的として作成する.

## 全体的な構造

Collinsの方法は, 多項式や有限次拡大体の帰納的な性質を利用しているため, それを意識して対象を扱う.

- 多項式は, $\mathrm{R}[x_1][x_2] \dots [x_r]$ という帰納的な構造を意識し, 任意の多項式は一意分解整域係数の1変数多項式とみなす. 
- 実代数的数は, 既約多項式$f_\alpha(x)$と分離区間$[a.b]$の組で表す.

## 射影段階

ここでは, 二つの関数を定義する.
- PROJ(F):
    + 入力: 一意分解整域 $U$ 係数1変数多項式環 $U[x]$ の有限部分集合F.
    + 出力: 一意分解整域 $U$ の有限部分集合proj(F).
- APROJ(F):
    + 入力: 一意分解整域 $U$ 係数1変数多項式環 $U[x]$ の有限部分集合F.
    + 出力: 一意分解整域 $U$ の有限部分集合aproj(F).

1変数多項式に対する様々な基本的な作用.

In [11]:
# 多項式微分(derivation)
def DER(f):
    return f.diff()

# 先頭係数(leading coefficient)
def LC(f):
    return f.lc()

# 先頭項(leading term)
def LT(f):
    return f.lt()

# 先頭単項式(leading monomial)
def LM(f):
    return f.lm()

# 先頭項の消去(reduction)
def RED(f):
    return f - LT(f)

# 多項式の次数(degree; ただし, degree(0)=-1)
def DEG(f):
    return f.degree()

# 主部分終結式係数全体(principal subresultant coefficient)
# 一つ一つ出力するほうがわかりやすいが, sagemathはsubresultantsモジュールしか備えておらず, そのためすべてを出力するように実装する.
def PSCs(f, g):
    """
    deg(f)>= 0, deg(g)>= 0, つまり, f != 0, g!= 0を仮定する.
    """
    if f == 0 or g == 0:
        return False
    psc_list = []
    subres = f.subresultants(g)
    for i in range(len(subres)):
        if DEG(subres[i])==i:
            psc_list.append(LC(subres[i]))
        else:
            psc_list.append(0)
    
    return psc_list
        

PROJ（F）の定義

In [12]:
def PROJ(F:set):
    """
    射影段階の実装を行う.
    Input: 一意分解整域R係数1変数多項式環R[x]の有限部分集合F
    Output: 一意分解整域Rの有限部分集合F
    """
    
    # FのReductionをすべて計算して, その集合をBとする.
    B = []
    for f in F:
        A = f
        while(A!=0):
            B.append(A)
            A = RED(A)
    B = list(set(B))
    
    # Bの先頭項係数全体をLとする.
    L = {LC(b) for b in B}
    
    # PSC_k(b, b')全体をS1とする.
    S1 = []
    for b in B:
        if DEG(b)>0:
            S1 = S1 + PSCs(b, DER(b))
    S1 = set(S1)
    
    # PSC_k(b1, b2)全体をS2とする.
    S2 = []
    for i in range(len(B)):
        for j in range(i+1,len(B)):
            S2 = S2 + PSCs(B[i],B[j])
    S2 = set(S2)
    
    return L|S1|S2

APROJ(F)の定義

In [13]:
def APROJ(F:set):
    
    # FのReductionをすべて計算して, その集合をBとする.
    B = []
    for f in F:
        A = f
        while(A!=0):
            B.append(A)
            A = RED(A)
    B = list(set(B))
    
    # Bの元の微分をすべて含み, Dとする.
    D = []
    for b in B:
        A = DER(b)
        while(DEG(A)>0):
            D.append(A)
            A = DER(A)
    
    # psc_k(d,d')をすべて含みPとする.
    P = []
    for d in D:
        P += PSCs(d,DER(d))
    P = set(P)
    
    return P|PROJ(F)

メソッドの確認.

In [14]:
f = PolynomialRing(QQ,"x")(x^3+2*x+1)
g = PolynomialRing(QQ,"x")(x^3+x)

PROJ({f,g}),APROJ({f,g})

({-2, -1, 0, 1, 2, 4, 5, 6, 12, 59},
 {-2, -1, 0, 1, 2, 4, 5, 6, 12, 36, 59, 72})

In [16]:
x,y,z = var("x, y, z")
f = PolynomialRing(PolynomialRing(PolynomialRing(QQ,"x"),"y"),"z")(z*y-x)
g = PolynomialRing(PolynomialRing(PolynomialRing(QQ,"x"),"y"),"z")(z*y+x)
PROJ(PROJ({f,g}))

{1, -2*x, -x, x}

## 底段階

- STRUM(f,g):
    + INPUT: f,g: $\mathrm{Q}[x]$の元
    - OUTPUT: fから構成されるSTURM列


In [121]:
def STRUM(f,g):
    if DEG(f)<=0:
        return 0
    S = [f, DER(f)]
    
    while(True):
        if S[-2] % S[-1] == 0:
            break
        else:
            S.append(-(S[-2] % S[-1]))
        
    return S

In [122]:
STRUM(PolynomialRing(QQ, "x")(x^2+1))

[x^2 + 1, 2*x, -1]

In [135]:
def COUNT(f,a,b):
    if DEG(f)<=0:
        return 0
    
    # スツルム列
    S = [f, f.diff()]
    
    while(True):
        if S[-2] % S[-1] == 0:
            break
        else:
            S.append(- (S[-2] % S[-1]))

    S_a = [s(a) for s in S]
    S_b = [s(b) for s in S]
    
    return var(S_a) - var(S_b)


In [102]:
NumberField?